<style>
    body, * {
        direction: rtl !important;
        text-align: right !important;
    }
</style>
# 🔄 פונקציות JSON ב-PySpark - מדריך מקיף
[PySpark JSON Functions with Examples](https://sparkbyexamples.com/pyspark/pyspark-json-functions-with-examples/)
## מבוא

ב-PySpark, פונקציות JSON מאפשרות לעבוד עם נתוני JSON בתוך DataFrames. פונקציות אלו עוזרות לך לנתח (parse), לתפעל ולחלץ נתונים מעמודות או מחרוזות JSON. ניתן להשתמש בפונקציות אלו גם להמרת JSON למבנה Struct, סוג Map ועוד.

במדריך זה נסקור את הפונקציות הנפוצות ביותר לעבודה עם JSON עם דוגמאות ב-Python.

---

## 📋 פונקציות JSON ב-PySpark

| פונקציה | תיאור |
|---------|-------|
| **from_json()** | ממירה מחרוזת JSON למבנה Struct או Map |
| **to_json()** | ממירה עמודות מסוג MapType או Struct למחרוזת JSON |
| **json_tuple()** | מחלצת אלמנטים מעמודת JSON ויוצרת עמודות חדשות |
| **get_json_object()** | מחלצת מחרוזת JSON על בסיס נתיב מעמודת JSON |
| **schema_of_json()** | יוצרת מחרוזת סכימה ממחרוזת JSON |

---

## 🔧 הגדרת הסביבה

ראשית, נייבא את הספריות הנדרושות וניצור SparkSession:

In [ ]:
from pyspark.sql import SparkSession, Row

# יצירת SparkSession
spark = SparkSession.builder.appName('PySpark-JSON-Functions').getOrCreate()

## 📝 יצירת DataFrame עם עמודה המכילה מחרוזת JSON

כדי להדגים את פונקציות JSON, ניצור DataFrame עם עמודה המכילה מחרוזת JSON:

In [ ]:
# הגדרת מחרוזת JSON
jsonString = '{"Zipcode":704,"ZipCodeType":"STANDARD","City":"PARC PARQUE","State":"PR"}'

# יצירת DataFrame
df = spark.createDataFrame(data = [(1, jsonString)], schema = ["id", "value"])

In [ ]:
# הצגת DataFrame
df.show(truncate=False)
df.printSchema()

---

# 🔨 פונקציות JSON - דוגמאות מעשיות

כעת נעבור על כל אחת מפונקציות JSON עם דוגמאות.

## 1️⃣ from_json() - המרת מחרוזת JSON ל-Map או Struct

הפונקציה `from_json()` משמשת להמרת מחרוזת JSON למבנה Struct או Map.

בדוגמה הבאה נמיר מחרוזת JSON לזוג מפתח-ערך מסוג Map:

In [ ]:
# המרת עמודת מחרוזת JSON לסוג Map
from pyspark.sql.types import MapType, StringType
from pyspark.sql.functions import from_json

df2 = df.withColumn("value", from_json(df.value, MapType(StringType(), StringType())))

In [ ]:
# הצגת הסכימה
df2.printSchema()

In [ ]:
# הצגת הנתונים
df2.show(truncate=False)

## 2️⃣ to_json() - המרת Map או Struct למחרוזת JSON

הפונקציה `to_json()` משמשת להמרת עמודות DataFrame מסוג MapType או Struct למחרוזת JSON.

כאן נשתמש ב-df2 שיצרנו בדוגמה הקודמת:

In [ ]:
from pyspark.sql.functions import to_json, col

# המרה חזרה למחרוזת JSON
df2.withColumn("value", to_json(col("value"))).show(truncate=False)
df2.printSchema()

## 3️⃣ json_tuple() - חילוץ אלמנטים מעמודת JSON

הפונקציה `json_tuple()` משמשת לשאילתה או חילוץ אלמנטים מעמודת JSON ויצירת תוצאה כעמודות חדשות:

In [ ]:
from pyspark.sql.functions import json_tuple

# חילוץ שדות ספציפיים מה-JSON
df.select(
    col("id"),
    json_tuple(col("value"), "Zipcode", "ZipCodeType", "City")
).toDF("id", "Zipcode", "ZipCodeType", "City").show(truncate=False)

## 4️⃣ get_json_object() - חילוץ ערך על בסיס נתיב JSON

הפונקציה `get_json_object()` משמשת לחילוץ מחרוזת JSON על בסיס נתיב (path) מעמודת JSON.

הפונקציה משתמשת בתחביר JSONPath (כמו `$.key`):

In [ ]:
from pyspark.sql.functions import get_json_object

# חילוץ ערך ספציפי באמצעות נתיב JSON
df.select(
    col("id"),
    get_json_object(col("value"), "$.ZipCodeType").alias("ZipCodeType")
).show(truncate=False)

## 5️⃣ schema_of_json() - יצירת מחרוזת סכימה

הפונקציה `schema_of_json()` משמשת ליצירת מחרוזת סכימה ממחרוזת JSON.

זה שימושי כאשר אתה רוצה לגלות את המבנה של נתוני JSON באופן אוטומטי:

In [ ]:
from pyspark.sql.functions import schema_of_json, lit

# יצירת מחרוזת סכימה מ-JSON
schemaStr = spark.range(1) \
    .select(schema_of_json(lit(
        '{"Zipcode":704,"ZipCodeType":"STANDARD","City":"PARC PARQUE","State":"PR"}'
    ))) \
    .collect()[0][0]

print(schemaStr)

---

## 📋 סיכום

במדריך זה למדנו על הפונקציות השונות לעבודה עם JSON ב-PySpark:

### 🎯 פונקציות מרכזיות:

1. **from_json()** - ממירה מחרוזת JSON למבנה Struct או Map, מאפשרת גישה מובנית לנתוני JSON

2. **to_json()** - ממירה עמודות MapType או Struct חזרה למחרוזת JSON, שימושי לייצוא נתונים

3. **json_tuple()** - מחלצת מספר שדות בו-זמנית מ-JSON ויוצרת עמודות חדשות, יעיל יותר מחילוץ שדות בודדים

4. **get_json_object()** - מחלצת ערך בודד מ-JSON באמצעות נתיב JSONPath, מושלם לגישה לשדות מקוננים

5. **schema_of_json()** - מגלה אוטומטית את הסכימה של נתוני JSON, חוסך זמן בהגדרת סכימות ידנית

### 💡 טיפים חשובים:

- השתמש ב-`from_json()` כאשר אתה צריך לעבוד עם נתוני JSON מובנים
- `json_tuple()` יעיל יותר מ-`get_json_object()` כאשר מחלצים מספר שדות
- השתמש ב-`schema_of_json()` כדי לגלות את המבנה של נתוני JSON לא מוכרים
- פונקציות JSON מטפלות בנתונים מקוננים ומבנים מורכבים

### 🔗 שימושים נפוצים:

- קריאת קבצי JSON
- עיבוד לוגים בפורמט JSON
- עבודה עם APIs שמחזירים JSON
- טרנספורמציה של נתונים בין פורמטים שונים

---

## 🔗 קישורים ומקורות נוספים

### מאמרים קשורים:
- [PySpark Parse JSON from String Column | TEXT File](https://sparkbyexamples.com/pyspark/pyspark-parse-json-from-string-column-text-file/)
- [PySpark Read JSON file into DataFrame](https://sparkbyexamples.com/pyspark/pyspark-read-json-file-into-dataframe/)
- [PySpark Read Multiple Lines (multiline) JSON File](https://sparkbyexamples.com/pyspark/pyspark-read-multiple-line-json-file/)
- [PySpark String Functions with Examples](https://sparkbyexamples.com/pyspark/pyspark-string-functions-with-examples/)
- [PySpark SQL Date and Timestamp Functions](https://sparkbyexamples.com/pyspark/pyspark-sql-date-and-timestamp-functions/)
- [PySpark Aggregate Functions with Examples](https://sparkbyexamples.com/pyspark/pyspark-aggregate-functions/)

### מקור המדריך:
- [PySpark JSON Functions with Examples - Spark By Examples](https://sparkbyexamples.com/pyspark/pyspark-json-functions-with-examples/)

---

## 🎓 למידה מהנה!

**Happy Learning!** 🚀

*מדריך זה תורגם ונערך בעברית מתוך [SparkByExamples.com](https://sparkbyexamples.com)*